# Teras Classificaiton Tutorial

## Introduction:

In this tutorial, we'll use Teras and its both API, default and LayerFlow for classification task.

**Model**: `TabTransformerClassifier`

**Dataset**: Adult Income dataset

## Data loading and Preprocessing

In [1]:
import pandas as pd
import numpy as np

adult_df = pd.read_csv("./datasets/adult_income_dataset.csv")
adult_df.head(3)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K


In [2]:
categorical_feats = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
numerical_feats = ['age', 'hours-per-week']

### Fill in the missing values

In [3]:
adult_df.replace("?", np.nan, inplace=True)
adult_df.loc[:, categorical_feats] = adult_df.loc[:, categorical_feats].fillna("missing")
adult_df.loc[:, numerical_feats] = adult_df.loc[:, numerical_feats].fillna(0)

In [4]:
adult_df.isna().sum().sum()

0

### Label Encode the target column

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
adult_df["income"] = le.fit_transform(adult_df["income"])

### No need to encode the Categorical Features in the dataset!!

Now you might wonder if we need to encode all the other categorical features in the dataset as well, since many of them contain string values, and we all know deep learning models require data to be in int/float format, right? Well, **No**, here you don't need to encode them.

**But Why?**

Because  here we're using `TabTransformer` model which is a **Transformer** based architecture for Classificaiton and Regression, it contains a `CategoricalFeatureEmbedding` layer which handles the categorical features, even if they contain string values. Just set the `encode` parameter to `True` and that layer will take care of encoding the string values.

Just to make it clear, here **encoding** means converting *string values to integers/floats* and **embedding** means converting those *encoded values to dense representations* of `embeddeing_dim` dimensionality.

### What's features metadata and why do we need it?

Some layers like `CategoricalFeatureEmbedding` that need to be able to differentiate between feature types and apply certain operations to relevant features, need a way for that differentiation.

And for that very purpose, we've introduced this concept of **Features Metadata** which is a dictionary that contains two sub dictionaries, one for categorical features and one for numerical features. 

The categorical features dictionary maps the categorical feature names to a tuple of feature index in the dataset and a list of unique values in that features i.e. `{feature_name: (feature_index, list_of_unique_values)}`. The list of unique values within a categorical feature is also sometimes referred to as the "*vocabulary*".
It can be accessed through `features_metadata["categorical"]`

The numerical features dictionary maps the numerical feature names to the feature index in the dataset i.e. `{feature_name: feature_index}`. 
It can be accessed through `features_metadata["numerical"]`

#### Why does the categorical features dictionary contains a list of unique values for each feature? 
The reason why categorical features dictionary contains a list of unqiue values for each feature is that, in case the user wants to encode the string/categorical values in the features, we want to have a lookup table to map these categorical values to their integer representations (here categorical values are mapped to their indices).

And we want to construct that lookup table on all of the categorical values, not just on values that exist within a batch of data which is what we have access to during training — since as you may guess the batch of data is likely to miss many of the categorical values that exist within the dataset and creating lookup table over just those values will result in unexpected and errornous results.

### We've got you covered!

No, you don't need to worry about constructing that features metadata dictionary yourself, `Teras` offers a handy utility function just for that purpose!

Just import the `get_features_metadata_for_embedding` function from the `teras.utils` module and pass it the dataset in `pandas DataFrame` format along with a list of categorical features names — if they don't exist in your dataset just leave it as `None`, and a list of numerial features names — again if they don't exist then leave it as `None`

In [10]:
from teras.utils import get_features_metadata_for_embedding

metadata = get_features_metadata_for_embedding(adult_df,
                                               categorical_features=categorical_feats,
                                               numerical_features=numerical_feats)

### Convert dataframe to tensorflow dataset

Now after you're done with preprocessing, you must convert your pandas DataFrame to a tensorflow dataset.

Here you should be mindful of two things:
1. If your dataset contains heterogenuos data, i.e. it contains numerical features and categorical features that are in string format, you must create a dictionary format tensorflow dataset. (Skip ahead to see how to do that!)
2. If you dataset contains homogenous data, i.e. it contains numerical features and categorical features that are already encoded to integers/floats, you can either create a dictionary format tensorflow dataset or the default array format tensorflow dataset. Though, in this case it is recommended to create an array format one.


Don't worry too much on how to do that, `Teras` makes it easy for you do create a dataset of either format.

Just import `dataframe_to_tf_dataset` function from `teras.utils` and pass it your dataframe, along with optional `target`, `batch_size` and `as_dict` parameters.

Now, remember the two different types of tensorflow datasets, well `as_dict` determines the type of tensorflow dataset that is created. If it is set to True, a dictionary format tensorflow dataset is created, otherwise an array foramt tensorflow dataset is created.

Since our dataset contains categroical features with string values, so we'll create a dictionary format tensorflow dataset

In [7]:
from teras.utils import dataframe_to_tf_dataset

adult_ds = dataframe_to_tf_dataset(dataframe=adult_df,
                                   target="income",
                                   as_dict=True)

And that's pretty much it for the preprocessing!

Notice, how much preprocessing code is cut short and is not needed for a Teras model, as compared to a typical machine leanring model such as SVM or tree based models.

Though it does need to be mentioned that, although the statement above holds true for most advanced architectures like transformer based models, some other architectures offered by `Teras` may require the categorical features to be encoded by user during preprocessing step, such as `DNFNetClassifier`, `NODEClassifier` etc.

## Importing and instantiating the model

Teras offers two different APIs for accesing and customizing the models to satiate different levels of accessibility and flexibility needs.

1. **Parametric API**: It is the default API and something you're already familiar with — you import the model class and specify the values for parameters that determine how the sub layers, models or blocks within the given model are constructed. 

    For instance, specify the `embedding_dim` parameter during instantiation of `TabTransformerClassifier` and that will be the dimensionality value used to construct/instatiate the `CategoricalFeatureEmbedding` layer. Simple enough, right?


2. **LayerFlow API**: It maximizes the flexbility while minimizing the interface. That may sound a bit contradictory at first, but let me explain. Here, instead of specifiy individual parameters specific to sub layers/models, the user instead specifies instances of those sub layers, models or blocks that are used in the given model architecture.

    For instance, instead of specifying the `embedding_dim` parameter value, the user specifies an instance of `CategoricalFeatureEmbedding` layer. 


    Now in this instance, we're just passing one parameter instead of another so it may not seem like much beneficial at first glance but let me highlight how it can immensely help depending on your use case:

    1. Since all you need to pass is an instance of layer, it can be any layer, there's no resitriction that it must be instance of `CategoricalFeatureEmbedding` layer — which means that you get complete control over not just customizing the existing layers offered by Teras but also you can design/create an Embedding layer of your own that can work in the place of the original  `CategoricalFeatureEmbedding` layer or any other layer for that matter. This is especially useful, if you're a desinging a new architecture and want to rapidly test out new modifications of the existing architectures by just plugging certain custom layers of your own in place of the default ones. Pretty cool, right?

    2. In many cases, to reduce the plethora of parameters and keep the most important ones, some parameters specific to sub-layers, models are not offered at the top level of the given architecture by the Parametric API, so if you need to tweak those parameters missing from the main model, you can use LayerFlow API and create an instance of that layer/model with desired parameters and pass it to the model.

    Now, some of you mega smart minded ones, might be thinking, what if there was a way to pass instances of certain layers but also just specify some parameter value for other layers and leave it to `Teras` to instantiate those layers, well I'm just like you fr and you can do just that with the LayerFlow API. Even though, the parameters specific to layers aren't inlcuded in the documentation of the LayerFlow version of those layers/models, but you can specify any parameter that is offered by the **Parametric API** in the **LayerFlow API** version of the model.
    
    For instance, say we just want to pass an instance of our custom Categorical Embedding layer but instead of specify an instance of the `Encoder` layer, we just want to modify the `dropout rate` for the `FeedForward` layer within it. Well since we know that the `TabTransformerClassifier`'s `Parametric` version exposes a `feed_forward_dropout` parameter, we can pass that keyword argument in the `LayerFlow` verson of the `TabTransformerClassifier`.

    Here's how you'd do it in the code:
    ```
    from teras.layerflow.models import TabTransformerClassifier

    custom_categorical_embedding = CustomCategoricalFeatureEmbedding()

    model = TabTransformerClassifier(categorical_feature_emebdding=custom_categorical_embedding,
                                    feed_forward_dropout=0.5)
    ```

    Now enough with theory, let's put things into practice!


### Using default API

In [11]:
from tensorflow import keras
from teras.models import TabTransformerClassifier

model = TabTransformerClassifier(num_classes=2,
                                features_metadata=metadata)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=["accuracy"])
history = model.fit(adult_ds, epochs=1)

48/48 [==============================] - 44s 559ms/step - loss: 0.4013 - accuracy: 0.8061


### Using LayerFlow API
Let's now use LayerFlow API and customize the `head` layer a bit before plugging it with the rest of the architecture.

In [13]:
from teras.layerflow.models import TabTransformerClassifier
# We've shortend the TabTransformer<LayerName> to TabT<LayerName> 
# to make it easier for imports and use.
# Let us know if that sounds confusing or if you've got other ideas for feedback!
from teras.layerflow.layers import TabTClassificationHead

# Say we want to use a custom hidden block for the TabTClassificationHead
# but want to use the default output layer
# Here, we'll just use a simple dense layer with relu activation
# as the hidden block
head_hidden_block = keras.layers.Dense(32, activation="relu")

# Note that, since TabTClassificaitonHead is from LayerFlow API,
# it doesn't direclty expose the num_classes parameter,
# but we know that the Parametric version of it does expose
# this parameter, hence we specify it's value so `Teras`
# takes care of creating an default output layer but with
# appropriate number of classes.
head = TabTClassificationHead(hidden_block=head_hidden_block,
                              num_classes=2)

model_lf = TabTransformerClassifier(features_metadata=metadata,
                                    head=head)
model_lf.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                 loss=keras.losses.BinaryCrossentropy(),
                 metrics="accuracy")
history = model_lf.fit(adult_ds, epochs=1)

/home/abaid/miniconda3/envs/ML/lib/python3.10/site-packages/numpy/core/numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


48/48 [==============================] - 42s 541ms/step - loss: 0.6360 - accuracy: 0.7256


## Wrappin it up!

And that wraps up our classification tutorial using Teras.
If you need more help, consult documentation, and other available resources and if that still leaves you with questions, feel free to raise an issue or email me khawaja.abaid@gmail.com

If you find `Teras` useful, please consider giving it a star on GitHub and sharing it with others! Thank you!